In [1]:
import requests
from bs4 import BeautifulSoup
import bs4
import os
import re
import tqdm


In [81]:
url = 'https://www.xiaohongshu.com/explore/6463763400000000070382d6?app_platform=android'

def fetch_url(url):
    headers = {
        'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/55.0.2883.87 UBrowser/6.2.4098.3 Safari/537.36',
        'Accept-Encoding': 'gzip, deflate',
        'cookie': 'your cookie here'
    }
    
    r = requests.get(url, headers = headers)
    return r

def download_img_from_url(url, i, base_dir='./'):
    # set proxy if do not need, just comment this line
    proxy = 'http://127.0.0.1:7891'
    r = requests.get(url, stream=True,
                     proxies={'http': proxy, 'https': proxy}, verify=False)
    ###############
    
    chunk_size = 1024
    content_size = int(r.headers['Content-Length'])
    
    os.makedirs(base_dir, exist_ok=True)
    with open(f'{base_dir}/{i}.jpg', 'wb') as f:
        pbar = tqdm.tqdm(r.iter_content(chunk_size=chunk_size), total=content_size/chunk_size)
        for data in pbar:
            f.write(data)
    f.close()
    print('下载成功')


In [68]:
r = fetch_url(url)
redirect_url = r.url
print('redirect_url:', redirect_url)

r = fetch_url(redirect_url)

# r.text
soup = BeautifulSoup(r.text, 'lxml')

with open('./text.xml','w') as f:
    f.writelines(soup.decode(pretty_print=True))


redirect_url: https://www.xiaohongshu.com/explore/6463763400000000070382d6?app_platform=android


In [79]:
############ legacy code #################

# water_mark_img_urls: list[bs4.element.Tag] = soup.find_all(name='div', attrs={'class': 'swiper-slide zoom-in'})
# wmius = []

# print('有水印的图片网址：')
# for i in range(len(water_mark_img_urls)):
#     img_url_text = water_mark_img_urls[0]['style']
#     l = img_url_text.find('url')
#     img_url = img_url_text[l+4:-2]
#     wmius.append(img_url)
#     print(img_url)
    
# r = wmius[0].find('.com')
# base_url = wmius[0][:r+4]


# scripts: list[bs4.element.Tag]=soup.find_all('script')
# text = scripts[8].text

#####################

image_base_url = 'https://sns-img-qc.xhscdn.com'

trace_id = re.findall(r'"traceId":"[a-zA-Z0-9]+"', r.text)

print('无水印的图片网址：')
ori_img_urls = []
for id in trace_id:
    wo_water_mark_img_url = image_base_url+ '/' + id.split(':')[-1].strip('"')
    ori_img_urls.append(wo_water_mark_img_url)
    print(wo_water_mark_img_url)


无水印的图片网址：
https://sns-img-qc.xhscdn.com/1000g0082gj64jc0ic0305os3fi57rq6iqdgshn8
https://sns-img-qc.xhscdn.com/1000g0082gj64jc0ic03g5os3fi57rq6ius63q70
https://sns-img-qc.xhscdn.com/1000g0082gj64jc0ic0605os3fi57rq6iodde5lo
https://sns-img-qc.xhscdn.com/1000g0082gj64jc0ic0005os3fi57rq6itj5oqlo
https://sns-img-qc.xhscdn.com/1000g0082gj64jc0ic0505os3fi57rq6i76aa3dg
https://sns-img-qc.xhscdn.com/1000g0082gj9m7p0i606g5os3fi57rq6is3i8gmg
https://sns-img-qc.xhscdn.com/1000g0082glhkkqqi206g5os3fi57rq6ims39uj8
https://sns-img-qc.xhscdn.com/1000g0082glhkkqqi20605os3fi57rq6im4vedvg
https://sns-img-qc.xhscdn.com/1000g0082glhkkqqi205g5os3fi57rq6ifnhkaj0
https://sns-img-qc.xhscdn.com/1000g0082glhkkqqi204g5os3fi57rq6inq2hj48
https://sns-img-qc.xhscdn.com/1000g0082glhkkqqi201g5os3fi57rq6idmbq02o
https://sns-img-qc.xhscdn.com/1000g0082glhkkqqi20305os3fi57rq6isotbndg
https://sns-img-qc.xhscdn.com/1000g0082glhkkqqi203g5os3fi57rq6iled8c3g
https://sns-img-qc.xhscdn.com/1000g0082glhkkqqi202g5os3fi57rq6iegip

In [ ]:
for i in range(len(ori_img_urls)):
    download_img_from_url(ori_img_urls[i], i, 'download_imgs')